In [67]:
import logging

def extract_bar_colors(image_path, num_colors=3, threshold=30):
    """
    Extract the main colors from the bars in a bar chart image,
    excluding black and white (or near-black/near-white) colors.

    Parameters:
        image_path (str): Path to the bar chart image file.
        num_colors (int): Number of distinct bar colors to extract.
        threshold (int): RGB distance threshold to filter black and white colors.

    Returns:
        list: A list of RGB tuples representing the main colors.
    """
    from PIL import Image
    import numpy as np
    from sklearn.cluster import KMeans
    from scipy.stats import mode

    # Set up logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

    # Open the image and convert it to RGB
    image = Image.open(image_path).convert("RGB")
    logging.info("Image loaded and converted to RGB.")

    # Convert image to numpy array and reshape for clustering
    pixels = np.array(image).reshape(-1, 3)
    logging.info("Image reshaped for clustering.")

    # Apply KMeans to identify dominant colors
    kmeans = KMeans(n_clusters=num_colors, random_state=42).fit(pixels)
    labels = kmeans.labels_
    clusters = kmeans.cluster_centers_.astype(int)
    logging.info("KMeans clustering completed.")

    # Use the most frequent color in each cluster as the representative color
    representative_colors = []
    for i in range(num_colors):
        cluster_pixels = pixels[labels == i]
        most_frequent_color  = [int(mode(cluster_pixels[:, j])[0]) for j in range(3)]
    
        representative_colors.append(most_frequent_color)
    
    logging.info("Representative colors from clusters: %s", representative_colors)

    # Filter out black and white colors
    filtered_colors = []
    for color in representative_colors:
        color = np.array(color, dtype=int).tolist()  # Ensure color is a list of integers
        distance_to_black = np.linalg.norm(np.array(color) - np.array([0, 0, 0]))
        distance_to_white = np.linalg.norm(np.array(color) - np.array([255, 255, 255]))

        if distance_to_black > threshold and distance_to_white > threshold:
            filtered_colors.append(tuple(color))

    logging.info("Filtered colors: %s", filtered_colors)

    return filtered_colors


In [51]:
# import logging

# def extract_bar_colors(image_path, top_colors=3, threshold=30):
#     """
#     Extract the most frequent colors from the bars in a bar chart image,
#     excluding black and white (or near-black/near-white) colors.

#     Parameters:
#         image_path (str): Path to the bar chart image file.
#         top_colors (int): Number of top most frequent colors to extract.
#         threshold (int): RGB distance threshold to filter black and white colors.

#     Returns:
#         list: A list of RGB tuples representing the most frequent colors.
#     """
#     from PIL import Image
#     import numpy as np
#     from collections import Counter

#     # Set up logging
#     logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

#     # Open the image and convert it to RGB
#     image = Image.open(image_path).convert("RGB")
#     logging.info("Image loaded and converted to RGB.")

#     # Convert image to numpy array and reshape for processing
#     pixels = np.array(image).reshape(-1, 3)
#     logging.info("Image reshaped for color counting. Total pixels: %d", pixels.shape[0])

#     # Count the occurrences of each color
#     color_counts = Counter(map(tuple, pixels))
#     logging.info("Color counting completed. Total unique colors: %d", len(color_counts))

#     # Filter out black and white colors
#     filtered_color_counts = {
#         color: count for color, count in color_counts.items()
#         if np.linalg.norm(np.array(color) - np.array([0, 0, 0])) > threshold and \
#            np.linalg.norm(np.array(color) - np.array([255, 255, 255])) > threshold
#     }
#     logging.info("Filtered out black and white colors. Remaining colors: %d", len(filtered_color_counts))

#     # Get the top most frequent colors
#     top_colors_list = sorted(filtered_color_counts.items(), key=lambda x: x[1], reverse=True)[:top_colors]
#     top_colors_rgb = [color for color, _ in top_colors_list]

#     logging.info("Top %d colors: %s", top_colors, top_colors_rgb)

#     return top_colors_rgb

In [54]:
# import logging

# def extract_bar_colors(image_path, top_colors=3, threshold=30, merge_distance=50):
#     """
#     Extract the most frequent colors from the bars in a bar chart image,
#     excluding black and white (or near-black/near-white) colors.
#     Similar colors are merged based on a distance threshold.

#     Parameters:
#         image_path (str): Path to the bar chart image file.
#         top_colors (int): Number of top most frequent colors to extract.
#         threshold (int): RGB distance threshold to filter black and white colors.
#         merge_distance (int): Maximum distance to merge similar colors.

#     Returns:
#         list: A list of RGB tuples representing the most frequent colors.
#     """
#     from PIL import Image
#     import numpy as np
#     from collections import Counter

#     # Set up logging
#     logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

#     # Open the image and convert it to RGB
#     image = Image.open(image_path).convert("RGB")
#     logging.info("Image loaded and converted to RGB.")

#     # Convert image to numpy array and reshape for processing
#     pixels = np.array(image).reshape(-1, 3)
#     logging.info("Image reshaped for color counting. Total pixels: %d", pixels.shape[0])

#     # Count the occurrences of each color
#     color_counts = Counter(map(tuple, pixels))
#     logging.info("Color counting completed. Total unique colors: %d", len(color_counts))

#     # Filter out black and white colors
#     filtered_color_counts = {
#         color: count for color, count in color_counts.items()
#         if np.linalg.norm(np.array(color) - np.array([0, 0, 0])) > threshold and \
#            np.linalg.norm(np.array(color) - np.array([255, 255, 255])) > threshold
#     }
#     logging.info("Filtered out black and white colors. Remaining colors: %d", len(filtered_color_counts))

#     # Merge similar colors based on merge_distance
#     merged_colors = {}
#     for color, count in filtered_color_counts.items():
#         found = False
#         for merged_color in list(merged_colors.keys()):
#             if np.linalg.norm(np.array(color) - np.array(merged_color)) <= merge_distance:
#                 merged_colors[merged_color] += count
#                 found = True
#                 break
#         if not found:
#             merged_colors[color] = count

#     logging.info("Colors merged based on distance. Remaining unique colors: %d", len(merged_colors))

#     # Get the top most frequent colors
#     top_colors_list = sorted(merged_colors.items(), key=lambda x: x[1], reverse=True)[:top_colors]
#     top_colors_rgb = [color for color, _ in top_colors_list]

#     logging.info("Top %d colors: %s", top_colors, top_colors_rgb)

#     return top_colors_rgb

In [25]:
def showcase_colors(colors):
    """
    Display the extracted colors in the terminal.

    Parameters:
        colors (list): A list of RGB tuples representing colors.
    """
    for i, color in enumerate(colors):
        r, g, b = color
        print(f"\033[48;2;{r};{g};{b}m    \033[0m Color {i + 1}: {color}")

In [60]:
# import matplotlib.pyplot as plt
# import numpy as np
# from PIL import Image
# from sklearn.cluster import KMeans

# def extract_bar_colors(image_path, num_colors=3):
#     """
#     Extract the main colors from the bars in a bar chart image.

#     Parameters:
#         image_path (str): Path to the bar chart image file.
#         num_colors (int): Number of distinct bar colors to extract.

#     Returns:
#         list: A list of RGB tuples representing the main colors.
#     """
#     # Open the image and convert it to RGB
#     image = Image.open(image_path).convert("RGB")

#     # Convert image to numpy array
#     img_array = np.array(image)

#     # Reshape the image array for clustering
#     pixels = img_array.reshape(-1, 3)

#     # Apply KMeans to identify dominant colors
#     kmeans = KMeans(n_clusters=num_colors, random_state=42)
#     kmeans.fit(pixels)

#     # Extract the RGB colors of the clusters
#     colors = kmeans.cluster_centers_.astype(int)

#     return [tuple(color) for color in colors]

In [29]:
showcase_colors(((1,2,3),(255,255,255)))

     Color 1: (1, 2, 3)
     Color 2: (255, 255, 255)


In [70]:
# Example usage
# Replace 'image_path' with the path to your bar chart image
image_path = "test_figs/bar_plot_test.jpg"
bar_colors = extract_bar_colors(image_path,  num_colors=4)
# bar_colors = extract_bar_colors(image_path, threshold=15, top_colors=10)
showcase_colors(bar_colors)
print("Filtered Bar Colors:", bar_colors)

2025-01-13 00:01:25,804 - INFO - Image loaded and converted to RGB.
2025-01-13 00:01:25,806 - INFO - Image reshaped for clustering.
2025-01-13 00:01:25,872 - INFO - KMeans clustering completed.
2025-01-13 00:01:25,895 - INFO - Representative colors from clusters: [[254, 254, 254], [27, 165, 0], [0, 4, 237], [248, 1, 0]]
2025-01-13 00:01:25,896 - INFO - Filtered colors: [(27, 165, 0), (0, 4, 237), (248, 1, 0)]


     Color 1: (27, 165, 0)
     Color 2: (0, 4, 237)
     Color 3: (248, 1, 0)
Filtered Bar Colors: [(27, 165, 0), (0, 4, 237), (248, 1, 0)]
